<a href="https://colab.research.google.com/github/Zuckmo/HR-analysis-for-Employee-Retention/blob/main/HR_Analysis_for_Employee_Retention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PACE STAGES
#PLAN
- Understanding the business scenario and problem
- Familiarize with HR dataset
1. step1 . Import
2. step2 Data Exploration (initial eda and data cleaning-missing value, redudandat, outliers)

#ANALYZE
- Data exploration (continue EDA)

#CONSTRUCT

#EXECUTE





#PLAN
the company initiates to improve satisfaction levels. They collected data from HR department. this project is to analyze the data collected by HR department and to buid a model that can predict whether the employee will leave the company or not.

if this project managed to predict the possibility of employee leaving, it might be possible to indentify some factors that contribute to their leaving. this will be helpful for the company to reduce cost of hiring new employees, interviewing employees and can increase employee retention.




In [ ]:
#IMPORTS

#for data manipulation
import numpy as np
import pandas as pd

#for data manipulation
import matplotlib.pyplot as plt
import seaborn as sns

#for display all the columns in dataframes
pd.set_option('display.max_columns', None)

#for data modelling
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from xgboost import plot_importance

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

#for metrics and helpful functions
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.tree import plot_tree

#for saving model
import pickle

In [ ]:
#load dataset

from google.colab import files
uploaded = files.upload()

In [ ]:

df = pd.read_csv("HR_comma_sep.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

RENAME COLUMNS
As a data cleaning step, rename the columns as needed. Standardize the column names so that they are all in snake_case, correct any column names that are misspelled, and make column names more concise as needed.

In [ ]:
df.columns

In [ ]:
df = df.rename(columns={'average_montly_hours':'average_monthly_hours', 'time_spend_company':'tenure', 'Work_accident':'work_accident', 'Department':'department'})

In [ ]:
df.columns

**CHECKING MISSING VALUES**

In [ ]:
df.isna().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
c = (3008/14999)*100
print(c)

this is 20% of duplicates


In [ ]:
df[df.duplicated()].head()

In [ ]:
#drop duplicates and save in new variable
df1 = df.drop_duplicates(keep='first') #if there is no keep the data will be error

In [ ]:
df1.tail()

**check outlier**
only tenure has outliers

In [ ]:
sns.boxplot(data=df1['tenure'])  # Replace 'df' with your DataFrame
plt.show()

it's valuabe to know how many rows that contain tenure outliers.

In [ ]:
#count 75%
percentile75 = df1['tenure'].quantile(0.75)
percentile25 = df1['tenure'].quantile(0.25)

#hitung iqr
iqr = percentile75 - percentile25

#count upper and lower limit
upper_limit = percentile75 + 1.5 * iqr
lower_limit =percentile25 - 1.5 * iqr

outliers = df1[(df1['tenure']>upper_limit) | (df1["tenure"] < lower_limit)]

print('number of data that contain outlier value of tenure :', len(outliers), "rows")

In [ ]:
df1.info()

In [ ]:
# Create a heatmap using seaborn
sns.heatmap(df1.corr(), annot=True, cmap="YlGnBu")

# Display the heatmap
plt.show()

#insight
It appears that employees are leaving the company as a result of poor management. Leaving is tied to longer working hours, many projects, and generally lower satisfaction levels. It can be ungratifying to work long hours and not receive promotions or good evaluation scores. There's a sizeable group of employees at this company who are probably burned out. It also appears that if an employee has spent more than six years at the company, they tend not to leave.


In [ ]:
df_en = df1.copy()

# Define the mapping of ordinal values to numerical values
ordinal_mapping = {
    'low': 1,
    'medium': 2,
    'high': 3}

#encode the salar as ordinal numeric category
df_en['salary']= df_en['salary'].map(ordinal_mapping)

In [ ]:
df_en.head()

In [ ]:
df_en = pd.get_dummies(df_en, columns=["department"], prefix=['department'])

In [ ]:
df_en.head()

In [ ]:
# Create a stacked bart plot to visualize number of employees across department, comparing those who left with those who didn't
# In the legend, 0 (purple color) represents employees who did not leave, 1 (red color) represents employees who left
pd.crosstab(df1['department'], df1['left']).plot(kind ='bar',color='mr')
plt.title('Counts of employees who left versus stayed across department')
plt.ylabel('Employee count')
plt.xlabel('Department')
plt.show()

since logistric regression is sensitive to outliers, so remove the outliers in tenure column


In [ ]:
df_logreg = df_en[(df_en['tenure'] >= lower_limit) & (df_en['tenure'] <= upper_limit)]
df_logreg

#isolate outcome variable("left")


In [ ]:
y = df_logreg['left']
y

In [ ]:
X = df_logreg.drop('left', axis=1)
X

In [ ]:
# Split the data into training set and testing set
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, stratify=y, random_state=100)

In [ ]:
# Construct a logistic regression model and fit it to the training dataset
log_clf = LogisticRegression(random_state=100, max_iter=500).fit(X_train, y_train)

In [ ]:
# Use the logistic regression model to get predictions on the test set
y_pred = log_clf.predict(X_test)


In [ ]:
#Create a confusion matrix to visualize the results of the logistic regression model.

# Compute values for confusion matrix
log_cm = confusion_matrix(y_test, y_pred, labels=log_clf.classes_)

# Create display of confusion matrix
log_disp = ConfusionMatrixDisplay(confusion_matrix=log_cm, display_labels=log_clf.classes_)


#plot confusion matrix
log_disp.plot(values_format='')

#display plot
plt.show()




The upper-left quadrant displays the number of true negatives. The upper-right quadrant displays the number of false positives. The bottom-left quadrant displays the number of false negatives. The bottom-right quadrant displays the number of true positives.

True negatives: The number of people who did not leave that the model accurately predicted did not leave.

False positives: The number of people who did not leave the model inaccurately predicted as leaving.

False negatives: The number of people who left that the model inaccurately predicted did not leave

True positives: The number of people who left the model accurately predicted as leaving

A perfect model would yield all true negatives and true positives, and no false negatives or false positives.

Create a classification report that includes precision, recall, f1-score, and accuracy metrics to evaluate the performance of the logistic regression model.

Check the class balance in the data. In other words, check the value counts in the left column. Since this is a binary classification task, the class balance informs the way you interpret accuracy metrics.

In [ ]:
df_logreg['left'].value_counts(normalize=True)

There is an approximately 83%-17% split. So the data is not perfectly balanced, but it is not too imbalanced. If it was more severely imbalanced, you might want to resample the data to make it more balanced. In this case, you can use this data without modifying the class balance and continue evaluating the model.

In [ ]:
target_names = ['predicted would not leave', 'predicted would leave']
print(classification_report(y_test, y_pred, target_names=target_names))

The classification report above shows that the logistic regression model achieved a precision of 79%, recall of 82%, f1-score of 80% (all weighted averages), and accuracy of 82%. However, if it's most important to predict employees who leave, then the scores are significantly lower.


#MODELLING APPROACH B : TREE-BASED MODEL
This approach covers implementation of Decision Tree and Random Forest.

In [ ]:
#Isolate the outcome
y = df_en['left']

In [ ]:
X = df_en.drop('left', axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=0)

**Desison tree - Round 1**

Construct a decision tree model and set up cross-validated grid-search to exhuastively search for the best model parameters.

In [ ]:
#instantiate model
tree = DecisionTreeClassifier(random_state=0)

#assign a dictionary of hyperparameters to search over
cv_params = {'max_depth':[4,6,8,None], 'min_samples_leaf': [2,5,1], 'min_samples_split':[2,4,6]}

#assign a dic of scoring metrics to capture
scoring = {'accuracy', 'precision', 'recall', 'f1', 'roc_auc'}

#instantiate GridSearch
tree1 = GridSearchCV(tree, cv_params, scoring=scoring, cv=4, refit='roc_auc')

Fit the decision tree model to the training data.


In [ ]:
%%time
tree1.fit(X_train, y_train)

Identify the optimal values for the decision tree parameters.

In [ ]:
tree1.best_params_

Identify the best AUC score achieved by the decision tree model on the training set.

In [ ]:
tree1.best_score_

This is a strong AUC score, which shows that this model can predict employees who will leave very well.

Next, you can write a function that will help you extract all the scores from the grid search.

In [ ]:
def make_results(model_name:str, model_object, metric:str):
    '''
    Arguments:
        model_name (string): what you want the model to be called in the output table
        model_object: a fit GridSearchCV object
        metric (string): precision, recall, f1, accuracy, or auc

    Returns a pandas df with the F1, recall, precision, accuracy, and auc scores
    for the model with the best mean 'metric' score across all validation folds.
    '''

    # Create dictionary that maps input metric to actual metric name in GridSearchCV
    metric_dict = {'auc': 'mean_test_roc_auc',
                   'precision': 'mean_test_precision',
                   'recall': 'mean_test_recall',
                   'f1': 'mean_test_f1',
                   'accuracy': 'mean_test_accuracy'
                  }

    # Get all the results from the CV and put them in a df
    cv_results = pd.DataFrame(model_object.cv_results_)

    # Isolate the row of the df with the max(metric) score
    best_estimator_results = cv_results.iloc[cv_results[metric_dict[metric]].idxmax(), :]

    # Extract Accuracy, precision, recall, and f1 score from that row
    auc = best_estimator_results.mean_test_roc_auc
    f1 = best_estimator_results.mean_test_f1
    recall = best_estimator_results.mean_test_recall
    precision = best_estimator_results.mean_test_precision
    accuracy = best_estimator_results.mean_test_accuracy

    # Create table of results
    table = pd.DataFrame()
    table = pd.DataFrame({'model': [model_name],
                          'precision': [precision],
                          'recall': [recall],
                          'F1': [f1],
                          'accuracy': [accuracy],
                          'auc': [auc]
                        })

    return table

Use the function just defined to get all the scores from grid search.

In [ ]:
# Get all CV scores
tree1_cv_results = make_results('decision tree cv', tree1, 'auc')
tree1_cv_results

All of these scores from the decision tree model are strong indicators of good model performance.

Recall that decision trees can be vulnerable to overfitting, and random forests avoid overfitting by incorporating multiple trees to make predictions. You could construct a random forest model next.

In [ ]:
# Instantiate model
rf = RandomForestClassifier(random_state=0)

# Assign a dictionary of hyperparameters to search over
cv_params = {'max_depth': [3,5, None],
             'max_features': [1.0],
             'max_samples': [0.7, 1.0],
             'min_samples_leaf': [1,2,3],
             'min_samples_split': [2,3,4],
             'n_estimators': [300, 500],
             }

# Assign a dictionary of scoring metrics to capture
scoring = {'accuracy', 'precision', 'recall', 'f1', 'roc_auc'}

# Instantiate GridSearch
rf1 = GridSearchCV(rf, cv_params, scoring=scoring, cv=4, refit='roc_auc')

Fit the random forest model to the training data.

In [ ]:
#%%time
rf1.fit(X_train, y_train) # --> Wall time: ~10min

Specify path to where you want to save your model.

In [ ]:
# Define a path to the folder where want to save the model
path = 'C:\DATASET\Dataset from google in Coursera\model HR'

Define functions to pickle the model and read in the model.

In [ ]:
def write_pickle(path, model_object, save_as:str):
    '''
    In:
        path:         path of folder where you want to save the pickle
        model_object: a model you want to pickle
        save_as:      filename for how you want to save the model

    Out: A call to pickle the model in the folder indicated
    '''

    with open(path + save_as + '.pickle', 'wb') as to_write:
        pickle.dump(model_object, to_write)

In [ ]:
def read_pickle(path, saved_model_name:str):
    '''
    In:
        path:             path to folder where you want to read from
        saved_model_name: filename of pickled model you want to read in

    Out:
        model: the pickled model
    '''
    with open(path + saved_model_name + '.pickle', 'rb') as to_read:
        model = pickle.load(to_read)

    return model

Use the functions defined above to save the model in a pickle file and then read it in.

In [ ]:
# Write pickle
write_pickle(path, rf1, 'hr_rf1')

In [ ]:
# Read pickle
rf1 = read_pickle(path, 'hr_rf1')

Identify the best AUC score achieved by the random forest model on the training set.

In [ ]:
# Check best AUC score on CV
rf1.best_score_

Identify the optimal values for the parameters of the random forest model.

In [ ]:
# Check best params
rf1.best_params_

In [ ]:
# Check best params
rf1.best_params_
# Check best params
rf1.best_params_
{'max_depth': 5,
 'max_features': 1.0,
 'max_samples': 0.7,
 'min_samples_leaf': 1,
 'min_samples_split': 4,
 'n_estimators': 500}

Collect the evaluation scores on the training set for the decision tree and random forest models.

In [ ]:
# Get all CV scores
rf1_cv_results = make_results('random forest cv', rf1, 'auc')
print(tree1_cv_results)
print(rf1_cv_results)

The evaluation scores of the random forest model are better than those of the decision tree model, with the exception of recall (the recall score of the random forest model is approximately 0.001 lower, which is a negligible amount). This indicates that the random forest model mostly outperforms the decision tree model.

Next, you can evaluate the final model on the test set.

Define a function that gets all the scores from a model's predictions.

In [ ]:
def get_scores(model_name:str, model, X_test_data, y_test_data):
    '''
    Generate a table of test scores.

    In:
        model_name (string):  How you want your model to be named in the output table
        model:                A fit GridSearchCV object
        X_test_data:          numpy array of X_test data
        y_test_data:          numpy array of y_test data

    Out: pandas df of precision, recall, f1, accuracy, and AUC scores for your model
    '''

    preds = model.best_estimator_.predict(X_test_data)

    auc = roc_auc_score(y_test_data, preds)
    accuracy = accuracy_score(y_test_data, preds)
    precision = precision_score(y_test_data, preds)
    recall = recall_score(y_test_data, preds)
    f1 = f1_score(y_test_data, preds)

    table = pd.DataFrame({'model': [model_name],
                          'precision': [precision],
                          'recall': [recall],
                          'f1': [f1],
                          'accuracy': [accuracy],
                          'AUC': [auc]
                         })

    return table

Now use the best performing model to predict on the test set.

In [ ]:
# Get predictions on test data
rf1_test_scores = get_scores('random forest1 test', rf1, X_test, y_test)
rf1_test_scores

The test scores are very similar to the validation scores, which is good. This appears to be a strong model. Since this test set was only used for this model, you can be more confident that your model's performance on this data is representative of how it will perform on new, unseeen data.

Feature Engineering
You might be skeptical of the high evaluation scores. There is a chance that there is some data leakage occurring. Data leakage is when you use data to train your model that should not be used during training, either because it appears in the test data or because it's not data that you'd expect to have when the model is actually deployed. Training a model with leaked data can give an unrealistic score that is not replicated in production.

In this case, it's likely that the company won't have satisfaction levels reported for all of its employees. It's also possible that the average_monthly_hours column is a source of some data leakage. If employees have already decided upon quitting, or have already been identified by management as people to be fired, they may be working fewer hours.

The first round of decision tree and random forest models included all variables as features. This next round will incorporate feature engineering to build improved models.

You could proceed by dropping satisfaction_level and creating a new feature that roughly captures whether an employee is overworked. You could call this new feature overworked. It will be a binary variable.

In [ ]:
# Drop `satisfaction_level` and save resulting dataframe in new variable
df2 = df_en.drop('satisfaction_level', axis=1)

# Display first few rows of new dataframe
df2.head()

In [ ]:
# Create `overworked` column. For now, it's identical to average monthly hours.
df2['overworked'] = df2['average_monthly_hours']

# Inspect max and min average monthly hours values
print('Max hours:', df2['overworked'].max())
print('Min hours:', df2['overworked'].min())

166.67 is approximately the average number of monthly hours for someone who works 50 weeks per year, 5 days per week, 8 hours per day.

You could define being overworked as working more than 175 hours per month on average.

To make the overworked column binary, you could reassign the column using a boolean mask.

df3['overworked'] > 175 creates a series of booleans, consisting of True for every value > 175 and False for every values ≤ 175
.astype(int) converts all True to 1 and all False to 0

In [ ]:
# Define `overworked` as working > 175 hrs/week
df2['overworked'] = (df2['overworked'] > 175).astype(int)

# Display first few rows of new column
df2['overworked'].head()